<a href="https://colab.research.google.com/github/akanison12/Sudo-code-2024/blob/main/240114_Understanding_LSTM_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download And Import Library

In [ ]:
!pip install kaggle
!kaggle datasets download -d iambestfeeder/10000-vietnamese-books
!pip install underthesea
!pip install vietnam-number
!pip install dask

Dataset URL: https://www.kaggle.com/datasets/iambestfeeder/10000-vietnamese-books
License(s): copyright-authors
 99% 535M/541M [00:07<00:00, 73.0MB/s]
100% 541M/541M [00:07<00:00, 76.6MB/s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00


In [ ]:
import re
import os
import zipfile
import csv
from underthesea import  text_normalize,word_tokenize
import pandas as pd
from vietnam_number import n2w
from dask import dataframe as dd
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar
import dask.bag as db
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
import torch.nn.functional as F


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
!git clone https://github.com/stopwords/vietnamese-stopwords

Cloning into 'vietnamese-stopwords'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 95 (delta 3), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (95/95), 40.25 KiB | 664.00 KiB/s, done.
Resolving deltas: 100% (31/31), done.


#Unzip and Load Dataset

In [ ]:

# Path to your zip file
zip_path = '/content/10000-vietnamese-books.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

#Make File
extracted_path = '/content/dataset'
print(os.listdir(extracted_path))

['output']


In [ ]:


def load_books(data_directory):
    texts = []
    titles = []
    authors = []

    files = os.listdir(data_directory)

    for file_name in files:
        file_path = os.path.join(data_directory, file_name)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read().strip()

            texts.append(content)

            title = file_name.replace('.txt', '').split('-')[:-1]
            author=file_name.replace('.txt', '').split('-')[-1]
            titles.append(title)
            authors.append(author)


    return pd.DataFrame({'Titles': titles, 'Texts': texts,'Authors':authors})



In [ ]:
data_directory = '/content/dataset/output'
dataset = load_books(data_directory)
dataset.head()

,Titles,Texts,Authors
0,"[Lobo, Sói xám vùng Kurumbo ]",Enexter Xeton Tomson\nLobo-Sói xám vùng Kurumb...,Enexter Xeton Tomson
1,[Ngưỡng cửa yêu thương ],Huỳnh Trung Chánh\nNgưỡng cửa yêu thương\nKính...,Huỳnh Trung Chánh
2,[Chuyện Trong Nhà ],Murakami Haruki\nChuyện Trong Nhà\nDịch giả: P...,Murakami Haruki
3,[Tự do cho một kiếp người ],Nguyễn Thị Ấm\nTự do cho một kiếp người\n(theo...,Nguyễn Thị Ấm
4,[Mùa Áo Vàng ],Từ kế Tường\nMùa Áo Vàng\nChương 1\nNhững ngọn...,Từ kế Tường


In [ ]:
dataset.shape

(10415, 3)

In [ ]:
datasub=dataset.sample(n=1000)


In [ ]:
datasub=pd.read_csv('/content/drive/MyDrive/ Sudo-ccode/W5/dataset_sub.csv')

In [ ]:
datasub

,Titles,Texts,Authors
0,['Tấm ảnh màu '],Cao Xuân Lý\nTấm ảnh màu\nKhông biết ai đã đặt...,Cao Xuân Lý
1,['Chuyến tàu đêm '],Slawomir Mrozek\nChuyến tàu đêm\nDịch giả: Lê ...,Slawomir Mrozek
2,"['Những bức thư từ Vúp', 'pơ', 'tan ']",Frederick Engels\nNhững bức thư từ Vúp-pơ-tan\...,Frederick Engels
3,['Tô canh bù ngót '],Huỳnh Trung Chánh\nTô canh bù ngót\nSuốt ngày ...,Huỳnh Trung Chánh
4,"['Các Nữ Thần Dưới Trướng Cờ Hai Bà Trưng ', '...",ĐỔ THỊ HẢO - MAI THỊ NGỌC CHÚC\nCác Nữ Thần Dư...,MAI THỊ NGỌC CHÚC
...,...,...,...
201,['Đứa Bé đi tìm cha '],Xuân Vũ\nĐứa Bé đi tìm cha\nMỞ ĐẦU\nThằng N...,Xuân Vũ
202,['Chim Trời '],Vũ Tú Nam\nChim Trời\nÔng tự thấy mình bay trê...,Vũ Tú Nam
203,['Xây Mộng '],Nguyễn Thế Hoàng\nXây Mộng\nSuốt buổi sáng chú...,Nguyễn Thế Hoàng
204,['Cọng Rêu Dưới Đáy Ao '],Võ Văn Trực\nCọng Rêu Dưới Đáy Ao\nChương I\nB...,Võ Văn Trực


#Preprocessing

In [ ]:
class Preprocessing:
    def __init__(self, stopword_file_path):
        # Load and read stopwords once
        self.stopwords = self.read_stopwords(stopword_file_path)

        # Compile regular expressions to improve processing speed
        self.source_pattern = re.compile(r'nguồn:\s*https?://\S+')
        self.quote_pattern = re.compile(r'&quot;')
        self.date_pattern = re.compile(r'(\d{1,2})[./-](\d{1,2})[./-]?(\d{2,4})?')
        self.number_pattern = re.compile(r'\d+')
        self.phone_number_pattern = re.compile(r'\b\d{10,11}\b')  # Phone numbers with 10 or 11 digits
        self.special_char_pattern = re.compile(r'[^\w\s]')
        self.whitespace_pattern = re.compile(r'\s+')

    def read_stopwords(self, file_path):
        # Read stopwords from the file
        with open(file_path, 'r', encoding='utf-8') as file:
            return set(line.strip() for line in file)

    def convert_date(self, match):
        # Convert dates in the format of day/month/year to words
        day, month, year = match.group(1), match.group(2), match.group(3) or ""
        day_in_words = n2w(day)
        month_in_words = n2w(month)
        year_in_words = n2w(year) if year else ""
        return f"{day_in_words} tháng {month_in_words}" + (f" năm {year_in_words}" if year_in_words else "")

    def safe_n2w(self, x):
        # If the number is a phone number, keep it as is
        if self.phone_number_pattern.fullmatch(x):
            return x  # Keep the phone number unchanged
        try:
            return n2w(x)  # Convert other numbers to words
        except:
            return x  # Return the original number if an error occurs

    def preprocessing_text(self, text):
        # Normalize and clean text
        text = text.lower().strip()

        # Remove extra spaces
        text = self.whitespace_pattern.sub(' ', text)

        # Remove 'Nguồn: HTTP'
        text = self.source_pattern.sub('', text)

        # Remove &quot; characters
        text = self.quote_pattern.sub('', text)

        # Handle dates with dots and slashes
        text = self.date_pattern.sub(self.convert_date, text)

        # Convert numbers to words (except for phone numbers)
        text = self.number_pattern.sub(lambda x: self.safe_n2w(x.group()), text)

        # Remove special characters
        text = self.special_char_pattern.sub('', text)

        # Normalize Vietnamese text
        text = text_normalize(text)

        # Tokenize the text
        words = word_tokenize(text, format="text")

        return words






#Test preprocessing

In [ ]:

stopword_file_path = '/content/vietnamese-stopwords/vietnamese-stopwords-dash.txt'
preprocessor = Preprocessing(stopword_file_path)
text = "Hãy gọi số điện thoại 0987654321 vào ngày 01-12-2022. Nguồn: http://example.com"



processed_text = preprocessor.preprocessing_text(text)


print(processed_text)

hãy gọi số điện_thoại 0987654321 vào ngày lẽ một tháng mười hai năm hai nghìn không trăm hai mươi hai


In [ ]:
file_path = '/content/vietnamese-stopwords/vietnamese-stopwords-dash.txt'
preprocessor = Preprocessing(file_path)

datasub['Processed_Text'] = datasub['Texts'].apply(preprocessor.preprocessing_text)


In [ ]:
datasub.to_csv('/content/drive/MyDrive/ Sudo-ccode/W5/dataset_sub_processing_1000samples.csv',index=False)

#Load data_clean and Build Vocabulary

In [ ]:
datasub=pd.read_csv('/content/drive/MyDrive/ Sudo-ccode/W5/dataset_sub_processing_1000samples.csv')

In [ ]:
train, test = train_test_split(datasub, test_size=0.2, random_state=42)

In [ ]:
class Vocabulary:
    """ The Vocabulary class is used to record words, which are used to convert
        text to numbers and vice versa.
    """
    def __init__(self):
        self.word2id = dict()

        self.pad_id = 0
        self.unk_id = 1
        self.sos_id = 2
        self.eos_id = 3

        self.word2id['<pad>'] = self.pad_id # Pad Token
        self.word2id['<unk>'] = self.unk_id # Unknown Token
        self.word2id['<s>'] = self.sos_id # start token
        self.word2id['</s>'] = self.eos_id # end token

        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __len__(self):
        return len(self.word2id)

    def lookup_tokens(self, word_indexes: list):
        """
        @param word_indexes (list(int))
        @return words (list(str))
        """
        return [self.id2word[word_index] for word_index in word_indexes]

    def add(self, word):
        """ Add word to vocabulary
        @param word (str)
        @return index (str): index of the word just added
        """
        if word not in self:
            word_index = self.word2id[word] = len(self.word2id)
            self.id2word[word_index] = word
            return word_index
        else:
            return self[word]

    def corpus_to_tensor(self, corpus, is_tokenized=False):
        """ Convert corpus to a list of indices tensor
        @param corpus (list(str) if is_tokenized==False else list(list(str)))
        @param is_tokenized (bool)
        @return indicies_corpus (list(tensor))
        """
        if is_tokenized:
            tokenized_corpus = corpus
        else:
            tokenized_corpus = self.tokenize_corpus(corpus)
        indicies_corpus = list()
        for document in tqdm(tokenized_corpus):
            indicies_document = torch.tensor(list(map(lambda word: self[word], document)), dtype=torch.long)
            indicies_corpus.append(indicies_document)

        return indicies_corpus

    def tensor_to_corpus(self, tensor):
        """ Convert list of indices tensor to a list of tokenized documents
        @param indicies_corpus (list(tensor))
        @return corpus (list(list(str)))
        """
        corpus = list()
        for indicies in tqdm(tensor):
            document = list(map(lambda index: self.id2word[index.item()], indicies))
            corpus.append(document)

        return corpus

    @staticmethod
    def tokenize_corpus(corpus):
        """Split the documents of the corpus into words
        @param corpus (list(str)): list of documents
        @return tokenized_corpus (list(list(str))): list of words
        """
        print("Tokenize the corpus...")
        tokenized_corpus = list()
        for document in tqdm(corpus):
            tokenized_document = ['<s>'] + re.findall(r'(\w+|[^\w\s]|\S+|\n)', document) + ['</s>']
            tokenized_corpus.append(tokenized_document)

        return tokenized_corpus

    @classmethod
    def from_documents(cls, documents):
        words = set(word for doc in documents for word in re.findall(r'\w+|\S|\n', doc))
        vocab = cls()
        for w in words:
            vocab.add(w)
        return vocab

    @classmethod
    def from_pretrained(cls, save_dir):
        import pickle
        """
        Initialize a new Vocabulary from a pre-trained vocabulary saved as a pickle file.

        Parameters:
        - vocab_file (str): Path to the pickle file containing the pre-trained vocabulary.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        with open(os.path.join(save_dir, "vocab.pkl"), 'rb') as file:
            pretrained_vocab = pickle.load(file)

        return cls.init_vocab_from_pretrained(pretrained_vocab)

    @staticmethod
    def init_vocab_from_pretrained(pretrained_vocab):
        """
        Initialize a new Vocabulary from a pre-trained vocabulary.

        Parameters:
        - pretrained_vocab (dict): A dictionary mapping words to their corresponding indices.

        Returns:
        - vocab (Vocabulary): Initialized Vocabulary object.
        """
        vocab = Vocabulary()

        # Copy the pre-trained word-to-index mapping to the new vocabulary
        vocab.word2id.update(pretrained_vocab)

        # Create the index-to-word mapping
        vocab.id2word = {v: k for k, v in vocab.word2id.items()}

        return vocab

    def save_pretrained(self, save_dir):
        import os
        import pickle
        """
        Save the vocabulary to a pickle file.

        Parameters:
        - pickle_file_path (str): Path to the pickle file where the vocabulary will be saved.
        """
        os.makedirs(save_dir, exist_ok=True)
        with open(os.path.join(save_dir, "vocab.pkl"), 'wb') as file:
            pickle.dump(self.word2id, file)

In [ ]:

document = train['Processed_Text'].tolist()
vocab = Vocabulary.from_documents(document)


print(f"Từ vựng có {len(vocab)} từ.")

Từ vựng có 109255 từ.


#Create DataLoader

In [ ]:


class GenerationDataset(Dataset):
    def __init__(self, documents, vocab, max_length=None):
        self.vocab = vocab
        self.sos_idx = vocab["<s>"]
        self.eos_idx = vocab["</s>"]
        self.pad_idx = vocab["<pad>"]
        self.documents = documents
        self.max_length = max_length
        self.tokenized_documents = self.vocab.tokenize_corpus(self.documents)
        self.tensor_data = self.vocab.corpus_to_tensor(self.tokenized_documents, is_tokenized=True)

    def __len__(self):
        return len(self.tensor_data)

    def __getitem__(self, idx):
        return self.tensor_data[idx]

    def shift_right(self, input_ids, pad_token=0):
        """
        Shifts the input tensor of batch of input ids to the right by one position.

        Args:
        - input_ids (torch.Tensor): Batch of input ids with shape (batch_size, sequence_length).
        - pad_token (int): Token used for padding. Defaults to 0.

        Returns:
        - torch.Tensor: Shifted input tensor with the same shape as input_ids.
        """
        # Create a tensor of the same shape filled with pad_token
        padding_column = torch.full_like(input_ids[:, :1], pad_token)

        # Concatenate the padding column to the left of the input_ids
        shifted_ids = torch.cat([padding_column, input_ids[:, :-1]], dim=-1)

        return shifted_ids


    def collate_fn(self, examples):
        examples = sorted(examples, key=lambda e: len(e), reverse=True)

        if self.max_length is not None:
            examples = [torch.cat([e[:self.max_length-1], torch.tensor([self.eos_idx])]) for e in examples]

        docs = [e for e in examples]
        # padding tokens
        input_ids = torch.nn.utils.rnn.pad_sequence(docs, batch_first=True, padding_value=self.pad_idx)
        labels = input_ids.clone()
        input_ids = self.shift_right(input_ids, pad_token=self.pad_idx)
        return {"inputs": input_ids, "labels": labels}

In [ ]:
train_documents = train['Processed_Text'].tolist()
val_documents = test['Processed_Text'].tolist()

In [ ]:
train_dataset = GenerationDataset(train_documents, vocab, max_length=512)
test_dataset = GenerationDataset(val_documents, vocab, max_length=512)

Tokenize the corpus...


100%|██████████| 164/164 [00:01<00:00, 142.13it/s]


Tokenize the corpus...


100%|██████████| 42/42 [00:00<00:00, 133.46it/s]


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=train_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=test_dataset.collate_fn)

#Build LSTM

In [ ]:
class LSTMForGeneration(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.padding_idx)
        self.batch_norm = nn.BatchNorm1d(config.hidden_size)
        self.lstm = nn.LSTM(config.hidden_size,
                            config.hidden_size,
                            num_layers=config.num_layers,
                            dropout=config.dropout,
                            batch_first=True)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, input_ids, labels=None):
        embeds = self.dropout(self.embedding(input_ids))
        bn_output = self.batch_norm(embeds.permute(0,2,1)).permute(0,2,1)
        output, (hidden, cell) = self.lstm(bn_output)

        logits = self.lm_head(self.dropout(output))
        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            labels = labels.to(logits.device)
            loss = loss_fct(logits.view(-1, logits.size(2)), labels.view(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, input_ids, max_length=100, temperature=1.0):
        self.to(input_ids.device)
        self.eval()

        current_length = input_ids.size(1)
        while current_length < max_length:
            logits, _ = self.forward(input_ids)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            current_length += 1

            # Check if </s> token is generated
            if next_token_id.item() == vocab.eos_id:
                break

        return input_ids

    def save_pretrained(self, save_dir):
        os.makedirs(save_dir, exist_ok=True)
        torch.save(self.config, os.path.join(save_dir, "config.pt"))
        torch.save(self.state_dict(), os.path.join(save_dir, "pytorch_model.pt"))

    @classmethod
    def from_pretrained(cls, saved_dir):
        config = torch.load(os.path.join(saved_dir, "config.pt"))
        model = cls(config)
        model.load_state_dict(torch.load(os.path.join(saved_dir, "pytorch_model.pt")))
        return model

In [ ]:
from dataclasses import dataclass

@dataclass
class LSTMConfig:
    vocab_size = len(vocab)
    hidden_size = 512
    padding_idx = 0
    dropout = 0.05
    num_layers = 2

config = LSTMConfig()

In [ ]:
model = LSTMForGeneration(config)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 116189895


In [ ]:
def train(model, train_dataloader, test_dataloader, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

    best_val_loss = float('inf')
    consecutive_no_improvement = 0
    gradient_accumulation_steps = 4

    for epoch in range(args.num_epochs):
        total_loss = 0.0
        model.train()

        for batch in (pbar:=tqdm(train_dataloader, desc=f"(train) Epoch {epoch}")):
            inputs, labels = batch['inputs'].to(device), batch['labels'].to(device)

            optimizer.zero_grad()

            logits, loss = model(inputs, labels)
            loss = loss / gradient_accumulation_steps
            loss.backward()

            if (epoch+1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()



            pbar.set_postfix(loss=loss.item())
            total_loss += loss.item()
            del inputs, labels, logits, loss

        average_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch}/{args.num_epochs}, Train Loss: {average_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for val_batch in (pbar:=tqdm(test_dataloader, desc=f"(val) Epoch {epoch}")):
                val_inputs, val_labels = val_batch['inputs'].to(device), val_batch['labels'].to(device)
                val_logits, val_loss_batch = model(val_inputs, val_labels)
                pbar.set_postfix(loss=val_loss_batch.item())
                val_loss += val_loss_batch.item()

        average_val_loss = val_loss / len(test_dataloader)
        print(f"Epoch {epoch}/{args.num_epochs}, Validation Loss: {average_val_loss:.4f}")

        # Early stopping check
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            consecutive_no_improvement = 0
            print(f"Save epoch {epoch} checkpoint")
            # torch.save({"state_dict": model.state_dict(), "vocab": vocab}, f"checkpoint_{epoch}.pt")
            model.save_pretrained(f"checkpoint_{epoch}")
            vocab.save_pretrained(f"checkpoint_{epoch}")
        else:
            consecutive_no_improvement += 1

        if consecutive_no_improvement >= args.patience:
            print(f"Early stopping after {epoch} epochs without improvement on validation set.")
            break

    print("Training complete.")

In [ ]:
class TrainingArguments:
    def __init__(self):
        self.num_epochs = 10
        self.patience = 3
        self.lr = 1e-3


training_args = TrainingArguments()

In [ ]:
train(model, train_dataloader, test_dataloader, args=training_args)

(train) Epoch 0: 100%|██████████| 21/21 [00:10<00:00,  1.92it/s, loss=2.9]


Epoch 0/10, Train Loss: 2.9007


(val) Epoch 0: 100%|██████████| 6/6 [00:01<00:00,  5.75it/s, loss=11.6]


Epoch 0/10, Validation Loss: 11.6040
Save epoch 0 checkpoint


(train) Epoch 1: 100%|██████████| 21/21 [00:10<00:00,  1.94it/s, loss=2.9]


...

Epoch 5/10, Validation Loss: 8.2765


(train) Epoch 6: 100%|██████████| 21/21 [00:11<00:00,  1.87it/s, loss=1.97]


Epoch 6/10, Train Loss: 1.9159


(val) Epoch 6: 100%|██████████| 6/6 [00:01<00:00,  5.66it/s, loss=7.81]

Epoch 6/10, Validation Loss: 8.2765
Early stopping after 6 epochs without improvement on validation set.
Training complete.


#Inference

In [ ]:
def tokenize(vocab, input_text):
    tokens = ['<s>'] + re.findall(r'\w+|\S|\n', input_text)
    input_ids = torch.tensor([vocab[word] for word in tokens], dtype=torch.long).unsqueeze(0)
    return input_ids

def generate_text(model, vocab, input_text, max_length=100, temperature=1.0):
    input_ids = tokenize(vocab, input_text).to("cuda")
    output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature)
    generated_tokens = [vocab.id2word[index.item()] for index in output_ids[0]]
    generated_text = ' '.join(generated_tokens[1:-1])  # Exclude <s> and </s>
    return generated_text



In [ ]:
# 1. load pretrained vocab
vocab = Vocabulary.from_pretrained("checkpoint_0")
model = LSTMForGeneration.from_pretrained("checkpoint_0")

<ipython-input-14-be05993fd7ce>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  config = torch.load(os.path.join(saved_dir, "config.pt"))
<ipython-input-14-be05993fd7ce>:5

In [ ]:
generate_text(model, vocab, "hôm nay tui đi học", max_length=512, temperature=.9)

'hôm nay tui đi học nôn_ọe sử_mỹ_pháp kiều_văn hẳnthiếu phô_trịch xuôi_gió điện_bàn ngây_tinh_khiết nghĩa_viên vội_chào tai_đâu copper nóbố cọc nhẹ_thúc nguyên_nam tộccộng_sản hiện_giờ tu_đà ma_rượt chu_rê lất bắc_phương phủ_lòng vãn_cảnh qlvnch_thời phụ_nữhọ tsogyel duyên_thanh_tịnh sáchhãy đại_thảm_kịch personne lê_cưa lữ_chia vĩnh_định_thần ghê_gớm_xẻ bao_gộp khinh_thị_tướng nữamấy acrehai quân_ân thiệu_định thực_thâm rồiném hội_đàm_mật cả_ngươi ria_hổ chiên_lạc quy_thuậnngày cađoàn chế_độ_sở_hữu_toàn_dân đắc_độ bạch_trèo đá_chìm sưu_thuế chùm câysáu tiến_xa tưởng_thể chiêu_hồi giảng_giáo_lý giông_quan cần_thiết_giác estonia audrey_antoine hotaru bước_nhảy_vọt kìakhánh lhomme vémột chân_thật tt biểu_diễn mìnhở mìnhnghe bullit có_học tay_chơi_bời khái_bước nguyên_thúy hồi_chuông kiếm_ăn chứng_quả phong_phú mùxoa mộtnhư ham_chăng binh_sĩ cháuông mau_kẻo như_ý thủ_lợi tịtchỉ mếncô vô_minh à_thế chiến_học huyền_nhận tó namphật_giáo moyen ngộ_giải hội_lý giọng_điệu sùng_mộ van_gogh khánh